## Task 1: Sentiment analysis of twitter and stockwits data

In [547]:
import json
from pprint import pprint
import matplotlib as mpl
import matplotlib.pyplot as pltfrom 

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import pandas as pd

from keras.layers import Flatten
from keras import regularizers
from sklearn.metrics.pairwise import cosine_similarity

### Loading data from JSON

In [548]:
with open('Microblog_Trainingdata.json') as data_file:    
    Microblog_Trainingdata = json.load(data_file)

for i in range(0,len(Microblog_Trainingdata)):
    Microblog_Trainingdata[i]['spans']=Microblog_Trainingdata[i]['spans'][0]
    Microblog_Trainingdata[i]['sentiment score']=float(Microblog_Trainingdata[i]['sentiment score'])
#pprint(Microblog_Trainingdata)
with open('Microblog_Trialdata.json') as data_file:   
    Microblog_Trialdata = json.load(data_file)
    

#pprint(data)
for i in range(0,len(Microblog_Trialdata)):
    Microblog_Trialdata[i]['spans']=Microblog_Trialdata[i]['spans'][0]
    Microblog_Trialdata[i]['sentiment score']=float(Microblog_Trialdata[i]['sentiment score'])

In [549]:
with open('Microblogs_Testdata.json') as data_file:   
    Microblogs_Testdata = json.load(data_file)
    
#pprint(Microblog_Testdata)

In [550]:
for i in Microblog_Trialdata:
    Microblog_Trainingdata.append(i)

In [551]:
def lower_keys(x):
    if isinstance(x, list):
        return [lower_keys(v) for v in x]
    elif isinstance(x, dict):
        return dict((k.lower(), v.lower() if isinstance(v, str) else v) for k, v in x.items())
    else:
        return x
Microblog_Trainingdata=lower_keys(Microblog_Trainingdata)

In [552]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [553]:
import string
def stringClean( inputString ) :
    return inputString.translate(str.maketrans('','',string.punctuation)).encode("ascii", errors="ignore").decode()

In [554]:
def removeStopWords(inputString):
    splitString = inputString.split()
    for word in stop:
        for s in splitString:
            if s == word:
                splitString.remove(s)
    return " ".join(splitString)

In [555]:
def calculateCosineSimilarity( pVal,gVal ):
    similarity = cosine_similarity(pVal,gVal)
    pNorm = np.linalg.norm(pVal)
    gNorm = np.linalg.norm(gVal)
    wt = pNorm/gNorm
    return wt * similarity #[0][0]

### Loading Testing and Testing data

In [556]:
maxlen = 0

X_train = []
X_trial=[]
Y_trial=[]
X_test = [Microblog['spans'] for Microblog in Microblogs_Testdata]
Y_train = []
Y_test = []
output=[]
for Microblog in Microblog_Trainingdata :
    X_train.append(removeStopWords(stringClean(Microblog['spans'])))
    Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
    output.append(Microblog['sentiment score'])
    if maxlen < len(Microblog['spans'].split()) :
        maxlen = len(Microblog['spans'].split())

#for Microblog in Microblog_Trialdata :
 #   X_train.append(Microblog['spans'])
  #  Y_train.append(0 if Microblog['sentiment score']<=0 else 1)
   # if maxlen < len(Microblog['spans'].split()) :
    #    maxlen = len(Microblog['spans'].split())


print(len(X_train)+len(X_test), len(Y_train))


2510 1710


### Selecting top 15000 words using tokenizer library in keras

In [557]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data = pad_sequences(sequences, maxlen)
data.shape

(1710, 25)

### LSTM Model

In [558]:
model = Sequential()
model.add(Embedding(15000, 32, input_length=maxlen))
#model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='linear'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.15))
#, kernel_regularizer=regularizers.l2(0.01), activity_regularizer=regularizers.l1(0.01)
model.add(Dense(4, activation='linear'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 25, 32)            480000    
_________________________________________________________________
lstm_19 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_47 (Dense)             (None, 4)                 132       
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 5         
Total params: 488,457
Trainable params: 488,457
Non-trainable params: 0
_________________________________________________________________


In [559]:
history = model.fit(data, np.array(Y_train), validation_split=0.2, epochs=15,verbose = 1)

Train on 1368 samples, validate on 342 samples
Epoch 1/15
1368/1368 [==============================] - 3s - loss: 1.2004 - acc: 0.4554 - val_loss: 0.6631 - val_acc: 0.6374
Epoch 2/15
1368/1368 [==============================] - 1s - loss: 0.6698 - acc: 0.6133 - val_loss: 0.6358 - val_acc: 0.6433
Epoch 3/15
1368/1368 [==============================] - 1s - loss: 0.6235 - acc: 0.6703 - val_loss: 0.6120 - val_acc: 0.6433
Epoch 4/15
1368/1368 [==============================] - 1s - loss: 0.5691 - acc: 0.7288 - val_loss: 0.5665 - val_acc: 0.6959
Epoch 5/15
1368/1368 [==============================] - 1s - loss: 0.4777 - acc: 0.8114 - val_loss: 0.4945 - val_acc: 0.7749
Epoch 6/15
1368/1368 [==============================] - 1s - loss: 0.3491 - acc: 0.8977 - val_loss: 0.4179 - val_acc: 0.7953
Epoch 7/15
1368/1368 [==============================] - 1s - loss: 0.2423 - acc: 0.9291 - val_loss: 0.4277 - val_acc: 0.8216
Epoch 8/15
1368/1368 [==============================] - 1s - loss: 0.1725 - ac

In [560]:
sequences = tokenizer.texts_to_sequences(np.array(X_train))
data = pad_sequences(sequences, maxlen)
predictions = model.predict(data)
for i in range(len(predictions)):
    print(Microblog_Trainingdata[i]['spans'],Microblog_Trainingdata[i]['sentiment score'],"%.3f" % predictions[i])

watching for bounce tomorrow 0.366 0.993
record number of passengers served in 2015 0.638 1.000
out $nflx -.35 -0.494 -0.485
looking for a strong bounce 0.46 0.998
very intrigued with the technology and growth potential 0.403 1.000
short worked 0.0 -0.651
overbought -0.296 -0.732
absolute garbage still up -0.546 -0.430
biggest market losers -0.438 -0.786
love this company long time. 0.408 0.989
$goog $googl would suck -0.398 -0.518
who won't pay anymore -0.349 -0.721
no edge offered 0.025 0.998
runs into the 50sma on the acquisition news 0.486 0.999
t can't go down 0.308 0.997
now seems like its helping the downtrend -0.372 -0.523
mastered their supply chain 0.461 0.999
most bullish stocks on twitter during this dip 0.408 1.000
big dumping -0.699 -0.282
trade continuing very nicely from yesterday 0.495 0.999
in from 3.61 0.306 0.982
stochastic overbought -0.385 -0.852
unusual call buying 0.336 1.000
reserves are in decline 0.279 1.000
all on a longer-term, technical swing long basis. 0

sell signal by $ndx internals -0.541 -0.653
bullshit -0.209 -0.431
$x +2.27% 0.144 0.983
this is what you get -0.007 0.150
another one to add, already happily holding 0.448 1.000
rsi is down at 20 0.418 0.998
$baba is going to be an interesting one 0.233 0.604
 0.315 0.681
buy the dip' 0.38 0.929
there's no reason why $googl can't get back to $1,000 0.288 0.996
short -0.453 -0.623
equity indexes working on a positive engulfing for march 0.308 1.000
highest call to put ratio 0.348 1.000
oil slid to a four-year low after -0.369 -0.609
sideways 0.0 -0.394
unusual call activity 0.421 1.000
green, lol... make-up to cover ugliness 0.242 0.999
slight green 0.328 0.999
pivotal sees 27% upside for alphabet 0.372 1.000
refuse -0.35 -0.257
profitable hard and software 0.559 0.968
$msft is opening up its #xboxlive online service 0.225 1.000
here we gooo 0.268 0.985
sold my 81 calls -0.312 0.492
at resistance right here -0.105 -0.321
holding 0.171 0.993
gevoâ€™s alcohol to jet fuel now eligible to 

the parties over -0.223 -0.531
up almost 0.378 0.770
$lei higher 0.388 0.952
breakout watch 0.352 1.000
adding to position here 0.685 0.999
brutal slide -0.044 0.255
looks good on the weekly chart 0.379 0.084
holding up relatively well compared to the broad market 0.394 0.999
covered the swing 0.021 1.000
bullish ma crossovers 0.457 0.999
top stocks in 2015 are ready to lead again in 2016 0.414 1.000
making new hod 0.442 0.999
defect -0.366 0.708
most bullish stocks on twitter during this dip 0.408 1.000
strong move today for the junior gold miners 0.75 1.000
tomorrow party time for the bears!! -0.524 -0.447
it is cheap. 0.183 -0.145
insiders are selling -0.527 0.032
why the coal & shipping sectors could rise again 0.418 1.000
small businesses have called $yelp a scam for years -0.45 0.939
weekly 120 puts +283% -0.35 -0.020
looking good 0.362 0.820
i may try a short -0.352 -0.369
equity indexes working on a positive engulfing for march 0.308 1.000
and it will continue 0.413 0.973
to co

In [561]:
df=pd.DataFrame.from_dict(Microblog_Trainingdata)

In [562]:
df['Predicted Sentiment Score']=predictions
df.head()

,cashtag,id,sentiment score,source,spans,Predicted Sentiment Score
0,$fb,719659409228451840,0.366,twitter,watching for bounce tomorrow,0.992637
1,$luv,719904304207962112,0.638,twitter,record number of passengers served in 2015,0.999988
2,$nflx,5329774,-0.494,stocktwits,out $nflx -.35,-0.485228
3,$dia,719891468173844480,0.460,twitter,looking for a strong bounce,0.997844
4,$plug,20091246,0.403,stocktwits,very intrigued with the technology and growth ...,0.999980


In [563]:
task1Dataframe=pd.DataFrame({"ID":df['id'],"Spans":df['spans'],"Cashtag":df['cashtag'],"Sentiment Score":df['sentiment score'],"Predicted Sentiment Score":df['Predicted Sentiment Score']})
task1Dataframe=task1Dataframe[['ID','Spans','Cashtag','Sentiment Score','Predicted Sentiment Score']]

In [564]:
#Convert dataframe to csv file
task1Dataframe.to_csv('Task1.csv',index=False)

### Computing Cosine score

In [508]:
from sklearn.metrics.pairwise import cosine_similarity

In [509]:
predictedVals = [pr[0] for pr in predictions ]

In [544]:
import numpy as np
output=np.array(output).reshape(1,-1)
predictedVals=np.array(predictedVals).reshape(1,-1)
similarity=cosine_similarity(predictedVals,output)
print(similarity)

[[ 0.81727216]]


In [511]:
np.random.seed(500)
temp =np.random.randint(0,len(Microblogs_Testdata),size=100)

In [512]:
newTestData=[]
for i in temp:
    newTestData.append(Microblogs_Testdata[i])

In [513]:
X_new_test=[(removeStopWords(stringClean(str(Microblog['spans'])))) for Microblog in newTestData]

In [516]:
sequences = tokenizer.texts_to_sequences(np.array(X_new_test))
data2 = pad_sequences(sequences, maxlen)
predictions2 = model.predict(data2)
for i in range(len(predictions2)):
    print(newTestData[i]['spans'],"%.3f" % predictions2[i])

congrats to whoever bought under 0.920
5 Best Analyst Rated Stocks 0.982
adding $expe and $pcln to the mix soon;$;I already have $CTRP and $trip 0.981
Iraq Boosts Oil Production to Record 0.976
Failed breakouts & failed setups;$;Limping on crutches into next week 0.514
 0.799
congrats to whoever bought under 0.920
$NFLX holding it's gain 0.949
$ADBE shorted, there was only 1% short float -0.770
Bottom Losses -0.807
Ron Johnson tried, and failed, to inject a bit of Apple culture into JC Penney -0.641
longed 0.799
over 225 million new Internet users are being added globally every year 0.982
AAPL could jump 50% partnering with Tsla -0.716
Most bullish stocks on Twitter during this dip 0.980
Board To Assess Alibaba-Softbank Proposal Further 0.799
Top Stocks in 2015 are Ready to Lead Again in 2016 0.982
Unusual Call Activity 0.982
Stochastic Sell Signal -0.650
Another Sell Rating -0.465
the resurgence in stock price today might mean that the company may emerge stronger than ever 0.981
consu

In [520]:
df2=pd.DataFrame(newTestData)

In [519]:
df2['Predicted Sentiment Score']=predictions2
df2.head(10)

,cashtag,id,source,spans,Predicted Sentiment Score
0,$PVA,38793629,stocktwits,congrats to whoever bought under,0.919769
1,$ORCL,710496429974888400,twitter,5 Best Analyst Rated Stocks,0.981834
2,$CTRP,709398139195641900,twitter,adding $expe and $pcln to the mix soon;$;I alr...,0.981309
3,$SPX,719493174167625700,twitter,Iraq Boosts Oil Production to Record,0.975918
4,$SBH,7867073,stocktwits,Failed breakouts & failed setups;$;Limping on ...,0.514061
5,$TNET,711745824884187100,twitter,,0.798638
6,$PVA,38793629,stocktwits,congrats to whoever bought under,0.919769
7,$NFLX,719530483898421200,twitter,$NFLX holding it's gain,0.948992
8,$ADBE,21174505,stocktwits,"$ADBE shorted, there was only 1% short float",-0.769696
9,$OASM,719699054645612500,twitter,Bottom Losses,-0.807218


In [545]:
df2.to_csv('newTestData.csv',index=False)

In [546]:
df2

,cashtag,id,source,spans
0,$PVA,38793629,stocktwits,congrats to whoever bought under
1,$ORCL,710496429974888400,twitter,5 Best Analyst Rated Stocks
2,$CTRP,709398139195641900,twitter,adding $expe and $pcln to the mix soon;$;I alr...
3,$SPX,719493174167625700,twitter,Iraq Boosts Oil Production to Record
4,$SBH,7867073,stocktwits,Failed breakouts & failed setups;$;Limping on ...
5,$TNET,711745824884187100,twitter,
6,$PVA,38793629,stocktwits,congrats to whoever bought under
7,$NFLX,719530483898421200,twitter,$NFLX holding it's gain
8,$ADBE,21174505,stocktwits,"$ADBE shorted, there was only 1% short float"
9,$OASM,719699054645612500,twitter,Bottom Losses
